In [50]:
# to increase the cell width of the notebook in the current browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
from readinputSPAST import READSPAST
from copy import deepcopy
from gurobipy import *
         
class GurobiSPAP():
    def __init__(self, filename):
        self.filename = filename
        r = READSPAST()
        r.read_file(self.filename)
        self.students = r.students
        self.projects = r.projects
        self.lecturers = r.lecturers
        
        self.sp = r.sp
        self.sp_copy = r.sp_copy
        self.sp_no_tie = r.sp_no_tie
        self.sp_no_tie_deletions = r.sp_no_tie_deletions
        self.plc = r.plc
        self.lp = r.lp        
        #self.stablematching = {'s1': 'p1', 's2': 'p5', 's3': 'p4', 's4': 'p2', 's5': '', 's6': '', 's7': 'p3'}
        #self.stablematching = {'s1': '', 's2': '', 's3': 'p2', 's4': 'p1'}
        self.J = Model("SPAP")
        

                
    def assignmentConstraints(self):
        #=============================================================================================================#
        # Create variables
        #=============================================================================================================#
        # =============================================== CONSTRAINT 4 ===============================================#
        # ...for each acceptable (student, project) pair, we create the binary variable xij and impose constraint 4   #
        #=============================================================================================================#
#         for student in self.sp:
#             self.sp[student].append(dict())
#             sumstudentvariables = LinExpr()
#             for project in self.sp[student][2]:
#                 if self.stablematching[student] == project:                    
#                     xij = self.J.addVar(lb=1.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name=student + " is assigned " + project)            
#                     self.sp[student][4][project] = xij
#                 else:
#                     # addVar(lb, ub, obj, vtype, name, column)
#                     xij = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name=student + " is assigned " + project)            
#                     self.sp[student][4][project] = xij            
#                 sumstudentvariables += xij 
#             # .. add constraint that a student can be assigned to at most one project
#             # addConstr(lhs, sense, rhs, name)
#             self.J.addConstr(sumstudentvariables <= 1, "Constraint for "+ student)
        
        for student in self.sp:                                    
            self.sp[student].append(dict()) # to store the binary variables for each project student finds acceptable
            sumstudentvariables = LinExpr()            
            for project in self.sp[student][2]:                
                # addVar(lb, ub, obj, vtype, name, column)
                xij = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name=student + " is assigned " + project)            
                self.sp[student][4][project] = xij            
                sumstudentvariables += xij 
            # .. add constraint that a student can be assigned to at most one project
            # addConstr(lhs, sense, rhs, name)
            self.J.addConstr(sumstudentvariables <= 1, "Constraint for "+ student)
        #=============================================================================================================#


            
        #=============================================================================================================#
        # =============================================== CONSTRAINT 5 ===============================================#
        # we loop through each project and each student that finds this project acceptable
        # then increment the corresponding student-project variable (xij)
        #=============================================================================================================#
        for project in self.plc:
            totalprojectcapacity = LinExpr()
            for student in self.sp:
                if project in self.sp[student][2]:
                    totalprojectcapacity += self.sp[student][4][project]        
            self.J.addConstr(totalprojectcapacity <= self.plc[project][1], "Total capacity constraint for "+ project)
        #=============================================================================================================#
        
                
        
        #=============================================================================================================#
        # =============================================== CONSTRAINT 6 ===============================================#
        # loop through each lecturer and each acceptable (student,project) pairs
        # if for an acceptable pair, the project is offered by lecturer, then increment
        # the totallecturercapacity with the corresponding (student,project) variable (xij)
        #=============================================================================================================#
        for lecturer in self.lp:
            totallecturercapacity = LinExpr()
            for student in self.sp:
                for project in self.sp[student][2]:
                    if lecturer == self.plc[project][0]:
                        totallecturercapacity += self.sp[student][4][project]
            self.J.addConstr(totallecturercapacity <= self.lp[lecturer][0], "Total capacity constraint for "+ lecturer) 
        #=============================================================================================================#
        
    
    
    
    
    #|--------------------------------------------------------------------------------------------------------------------------|#
    #|                                                                                                                          |#
    #| For an arbitrary acceptable pair (s_i, p_j), we define all the relevant terms to ensure (s_i, p_j) does not block M..    |#
    #|                                                                                                                          |#
    #|--------------------------------------------------------------------------------------------------------------------------|#
    
    
    #=============================================================================================================#
    # we define thetaij :::::: 
    # If thetaij = 1, s_i is either unmatched in M or strictly prefers p_j to M(s_i) or is indifferent between them
    #=============================================================================================================#
    def theta(self, student, project):
        thetaij = LinExpr()
        sumSij = LinExpr() 
        xij = self.sp[student][4][project]
        indexproject = self.sp[student][2][project][0] # get the rank of project on student's preference list
        # now we loop through each project (p_j') that student strictly prefers to project (p_j):: p_j not inclusive
        for tie in self.sp[student][1][:indexproject]:
            for pjprime in tie:       
                sumSij += self.sp[student][4][pjprime]
        # sumSij = quicksum(self.studentdict[student][1][pjprime] for pjprime in studentPreference[:indexproject+1])        
        # thetaij += 1 - sumSij        
        thetaij.addConstant(1.0)
        thetaij.add(sumSij, -1)
        thetaij.add(xij, -1)
        # print(G.theta('s1', 'p3')) --- to verify the validity of quicksum on sumSij ----- working.        
        # <gurobi.LinExpr: 1.0 + -1.0 s1 is assigned p3>
        return thetaij      
    #=============================================================================================================#
    
    
    #=============================================================================================================#
    # =============================================== CONSTRAINT 7 ===============================================#
    # we define alpha_j to be a binary variable that corresponds to the occupancy of p_j in M
    # if p_j is undersubscribed in M then we enforce alpha_j to be 1
    #=============================================================================================================#    
    def alpha(self, project):        
        alphaj = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name=project+" is undersubscribed")                
        capacity = self.plc[project][1]           # c_j
        projectoccupancy = LinExpr()
        for student in self.sp:
            if project in self.sp[student][2]:
                projectoccupancy += self.sp[student][4][project]
        #projectoccupancy = quicksum(self.studentdict[student][1][project] for student in self.studentdict if project in self.studentdict[student][0])
        self.J.addConstr(capacity*alphaj >= (capacity - projectoccupancy), "constraint 7")
        return alphaj
    #=============================================================================================================# 
    
    # =============================================== CONSTRAINT 8 ==============================================#
    # we create a binary variable betak that corresponds to the occupancy of l_k in M. 
    # If l_k is undersubscribed in M, we enforce betak = 1
    #=============================================================================================================#
    def beta(self,student,project):                
        lecturer = self.plc[project][0]              # l_k
        lecturercapacity = self.lp[lecturer][0]    # d_k                
        betak = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name= lecturer + " is undersubscribed")
        lectureroccupancy = LinExpr()
        for pro in self.lp[lecturer][2]:
            for tie in self.lp[lecturer][2][pro]:
                for stud in tie:         
                    lectureroccupancy += self.sp[stud][4][pro]
        #lectureroccupancy = quicksum(self.studentdict[student][1][pjprime] for student in self.studentdict for pjprime in lecturerPreference if pjprime in self.studentdict[student][0])                
        self.J.addConstr((lecturercapacity*betak) >= (lecturercapacity - lectureroccupancy), "constraint 8")             
        return betak
    #=============================================================================================================#
    
    
    # =============================================== CONSTRAINT 10 ==============================================#
    # we create a binary variable etak 
    # if l_k is full in M, we enforce etak = 1
    #=============================================================================================================#
    def eta(self,student,project):        
        lecturer = self.plc[project][0]              # l_k
        dk = self.lp[lecturer][0]                   # d_k
        lecturerpreference = self.lp[lecturer][1]  # L_k
        etak = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name= lecturer+ " is full")
        lecturercapacity = LinExpr()     

        for proj in self.lp[lecturer][2]:
            for tie in self.lp[lecturer][2][proj]:
                for stud in tie:         
                    lecturercapacity += self.sp[stud][4][proj]    
        
        lecturercapacity.addConstant(1.0)                
        self.J.addConstr((dk*etak) >= (lecturercapacity - dk), "constraint 10")             
        return etak
    #=============================================================================================================#
    
    
    
    # =============================================== CONSTRAINT 11 ==============================================#
    # we create a binary variable deltaik 
    # if s_i \in M(l_k) or l_k strictly prefers s_i to a worst student in M(l_k) or is
    # indifferent between them, we enforce deltaik = 1
    #=============================================================================================================#
    def delta(self,student,project):        
        lecturer = self.plc[project][0]              # l_k
        dk = self.lp[lecturer][0]    # d_k
        lecturerpreference = self.lp[lecturer][1]  # L_k
        deltaik = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name= lecturer+ " prefers " + student + " to his worst assignee")
        lecturercapacity = LinExpr()     
        
#         for tie in lecturerpreference:
#             for student in tie:
#                 for project in self.sp[student][2]:
#                     if lecturer == self.plc[project][0]:            
#                         lecturercapacity += self.sp[student][4][project]
        for proj in self.lp[lecturer][2]:
            for tie in self.lp[lecturer][2][proj]:
                for stud in tie:         
                    lecturercapacity += self.sp[stud][4][proj]
      
        index = 0
        #found = False
        while True:
            if student in lecturerpreference[index]:
                break 
            index += 1
        Dik = lecturerpreference[:index]
        
        lectureroccupancy = LinExpr()        
        
        for ti in Dik:
            for st in ti:
                for pr in self.sp[st][2]:
                    if self.plc[pr][0] == lecturer:
                        lectureroccupancy += self.sp[st][4][pr]
        
        lecturercapacity.add(lectureroccupancy, -1)        
        self.J.addConstr((dk*deltaik) >= (lecturercapacity), "constraint 11")             
        return deltaik
    #=============================================================================================================#
    
    
    
    #=============================================================================================================#
    # =============================================== CONSTRAINT 13 ===============================================#
    # we define gamma_j to be a binary variable that corresponds to the occupancy of p_j in M
    # if p_j is full in M then we enforce gamma_j to be 1
    #=============================================================================================================#    
    def gamma(self, project):        
        gammaj = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name=project+" is full")                
        cj = self.plc[project][1]           # c_j
        projectoccupancy = LinExpr()
        for student in self.sp:
            if project in self.sp[student][2]:
                projectoccupancy += self.sp[student][4][project]
        self.J.addConstr(cj*gammaj >= ((1+projectoccupancy)-cj), "constraint 13")
        return gammaj
    
    #=============================================================================================================# 
                                         
    # =============================================== CONSTRAINT 14 ==============================================#
    # we create a binary variable lambdaijk 
    # if l_k strictly prefers s_i to a worst student in M(p_j) or is
    # indifferent between them, we enforce lambdaijk = 1
    #=============================================================================================================#
    def Lambda(self,student,project):        
        lecturer = self.plc[project][0]              # l_k
        cj = self.plc[project][1]
        lambdaijk = self.J.addVar(lb=0.0, ub=1.0, obj=0.0, vtype=GRB.BINARY, name= lecturer+ " prefers " + student + " to his worst student in M("+project+")")
        projectcapacity = LinExpr() 
        projectpreference = self.lp[lecturer][2][project]    # L_k^j
        for tie in projectpreference:
            for stud in tie:
                projectcapacity += self.sp[stud][4][project]
                
        index = 0
        while True:  # find the position of the tie containing student
            if student in projectpreference[index]:
                break
            index += 1
        Tijk = projectpreference[:index]
        
        projectoccupancy = LinExpr()
        for ti in Tijk:
            for st in ti:
                projectoccupancy += self.sp[st][4][project]
                
        projectcapacity.add(projectoccupancy,-1)        
        #lectureroccupancy = quicksum(self.studentdict[student][1][pjprime] for student in self.studentdict for pjprime in lecturerPreference if pjprime in self.studentdict[student][0])                
        self.J.addConstr((cj*lambdaijk) >= (projectcapacity), "constraint 14")             
        return lambdaijk
    #=============================================================================================================#
    
                                    
                                    
    #=============================================================================================================#
    # =============================================== CONSTRAINT 9, 12 AND 15 ===================================#
    # we enforce constraints to avoid blocking pair of type 2a, 2b and 2c for each acceptable (student, project) pairs
    #=============================================================================================================#
    def avoidblockingpair(self):        

        # for all acceptable (student, project) pairs
        # for better complexity ::: is there a way to only check those pairs that could block the final matching?
        for student in self.sp:             
            for project in self.sp[student][2]:             
                thetaij = self.theta(student, project)
                alphaj = self.alpha(project)
                betak = self.beta(student, project)
                etak = self.eta(student, project)
                deltaik = self.delta(student, project)
                gammaj = self.gamma(project)
                lambdaijk = self.Lambda(student, project)                
                ## ---- blocking pair 2a -----
                self.J.addConstr(thetaij + alphaj + betak <= 2, "constraint 9 - avoid blocking pair 2a")
                ## ----- blocking pair 3b -----
                self.J.addConstr(thetaij + alphaj + etak + deltaik <= 3, "constraint 12 - avoid blocking pair 2b")              
                ## ----- blocking pair 3c -----
                self.J.addConstr(thetaij + gammaj + lambdaijk <= 2, "constraint 14 - avoid blocking pair 2c")  
                
    #=============================================================================================================#
                                    
                                    
    #=============================================================================================================#
    # =============================================== CONSTRAINT 14 ==============================================#
    # we maximize the objective function
    #=============================================================================================================#
    def objfunctionConstraints(self):        
        # finally we add the objective function to maximise the number of matched student-project pairs
        Totalxijbinaryvariables = LinExpr()
        for student in self.sp:            
            for project in self.sp[student][2]:
                Totalxijbinaryvariables += self.sp[student][4][project]
        
        #setObjective(expression, sense=None)
        self.J.setObjective(Totalxijbinaryvariables) 

                                    
                                    
directory = 'experiments/1/'
output = directory+'IPoutput.txt'
with open(output, 'w') as O:
    for i in range(100,1100, 100):
        input_file = directory + 'instance_' + str(i) + '.txt'
        G = GurobiSPAP(input_file)
        try:
            G.assignmentConstraints()
            G.objfunctionConstraints() 
            G.avoidblockingpair()        
            G.J.optimize()
            
            count = 1
            #M = {}
            while count <= len(G.sp):
                student = 's'+str(i)
                #M[student] = []
                for project in G.sp[student][2]:            
                    a = G.J.getVarByName(student + " is assigned " + project)                   
                    if a.x == 1.0:
                        continue
                        #M[student].append(project)
                count += 1
        
            O.write('instance_' + str(i) + ': Has a super-stable matching' + ' \n')
        except GurobiError:
            O.write('instance_' + str(i) + ': No super-stable matching exists' + ' \n')
    O.close()
                                    

                                    
# G = GurobiSPAP("instances/tie-9.txt")
# try:
#     G.assignmentConstraints()
#     G.objfunctionConstraints() 
#     G.avoidblockingpair()        
#     G.J.optimize()   

#     i = 1
#     M = {}
#     while i <= len(G.sp):
#         student = 's'+str(i)
#         M[student] = []
#         for project in G.sp[student][2]:            
#             a = G.J.getVarByName(student + " is assigned " + project)                   
#             if a.x == 1.0:
#                 M[student].append(project)
#         i += 1

    

# except GurobiError:1
#     print('No super-stable matching exists')                               
                                    


Optimize a model with 45170 rows, 35000 columns and 5372621 nonzeros
Variable types: 0 continuous, 35000 integer (35000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 96 rows and 94 columns (presolve time = 5s) ...
Presolve removed 1861 rows and 1713 columns (presolve time = 10s) ...
Presolve removed 1977 rows and 1833 columns (presolve time = 15s) ...
Presolve removed 2022 rows and 2756 columns (presolve time = 21s) ...
Presolve removed 3086 rows and 2816 columns (presolve time = 26s) ...
Presolve removed 3086 rows and 2816 columns (presolve time = 30s) ...
Presolve removed 3086 rows and 2816 columns (presolve time = 35s) ...
Presolve removed 3086 rows and 2816 columns (presolve time = 41s) ...
Sparsify removed 4338430 nonzeros (95%)
Presolve removed 3097 rows and 39965 columns
Presolve time: 41.78s
Presolved: 42073 rows, 34307 columns, 242587 nonze

In [ ]:
    G = GurobiSPAP("instances/tie-9.txt")
    try:
        G.assignmentConstraints()
        G.objfunctionConstraints() 
        G.avoidblockingpair()        
        G.J.optimize()   
#     print()
#     print("Gurobi optimizer is done!")
#     print("..printing matching..")
#     print(len(G.sp), G.sp)  
#     l = 'l1'
#     s = 's4'
#     p = 'p1'
#     print(G.J.getVarByName(l+ " prefers " + s + "to his worst assignee"))
#     print(G.J.getVarByName(p+ " is undersubscribed"))
#     print(G.J.getVarByName(l+ " is undersubscribed"))
#     print(G.J.getVarByName(l+ " prefers " + s + " to his worst assignee"))
#     print(G.J.getVarByName(l+ " prefers " + s + " to his worst student in M("+p+")"))


#     print(G.J.getVarByName(s + " is assigned " + p) )
#     print(G.theta('s4','p1'))

    
    i = 1
    M = {}
    while i <= len(G.sp):
        student = 's'+str(i)
        M[student] = []
        for project in G.sp[student][2]:            
            a = G.J.getVarByName(student + " is assigned " + project)                   
            if a.x == 1.0:
                M[student].append(project)
                print(student, ' ', project)
        i += 1
    print()
    print(M)

    

except GurobiError:
    print('Error reported')